In [1]:
import pandas as pd
import numpy as np

In [2]:
ba=pd.read_csv(r"/Users/kglobaltechnologies/Downloads/ba_reviews.csv")
ba.head()

,header,author,date,place,content,aircraft,traveller_type,seat_type,route,date_flown,recommended,trip_verified,rating,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,entertainment
0,service was mediocre at best,Gary Storer,03/10/2023,United Kingdom,"Just returned from Chicago, flew out 10 days ...",A380,Couple Leisure,Economy Class,Chicago to Manchester via Heathrow,01/10/2023,no,Not Verified,2,2,3,1,2,2,-1
1,BA standards continue to decline,A Jensen,02/10/2023,United Kingdom,BA standards continue to decline every time ...,A320,Business,Business Class,London Heathrow to Munich,01/09/2023,no,Verified,2,2,1,2,1,1,-1
2,"won the race to the bottom""",John Rockett,02/10/2023,United Kingdom,Awful. Business class check in queue just as...,A320,Couple Leisure,Business Class,Heathrow to Istanbul,01/09/2023,no,Not Verified,2,2,3,2,1,1,-1
3,Not a reliable airline,Tatiana Bobrovskaya,02/10/2023,United Kingdom,Not a reliable airline. You cannot trust the...,A320,Business,Economy Class,London to Geneva,01/10/2023,no,Verified,3,4,4,2,1,1,-1
4,Very disappointed,Tom Slowbe,28/09/2023,United States,"The airplanes and the lounges are worn out, o...",777-300 and A320,Couple Leisure,First Class,Dallas to Dubrovnik via Heathrow,01/09/2023,no,Verified,1,1,4,4,3,2,3


In [3]:
ba.columns

Index(['header', 'author', 'date', 'place', 'content', 'aircraft',
       'traveller_type', 'seat_type', 'route', 'date_flown', 'recommended',
       'trip_verified', 'rating', 'seat_comfort', 'cabin_staff_service',
       'food_beverages', 'ground_service', 'value_for_money', 'entertainment'],
      dtype='object')

In [4]:
ba.dtypes

header                 object
author                 object
date                   object
place                  object
content                object
aircraft               object
traveller_type         object
seat_type              object
route                  object
date_flown             object
recommended            object
trip_verified          object
rating                  int64
seat_comfort            int64
cabin_staff_service     int64
food_beverages          int64
ground_service          int64
value_for_money         int64
entertainment           int64
dtype: object

In [5]:
ba.shape

(1324, 19)

In [6]:
rating_cols=['seat_comfort','cabin_staff_service',
             'food_beverages','ground_service','value_for_money','entertainment','rating']
for col in rating_cols:
    ba[col]=pd.to_numeric(ba[col],errors='coerce')
    ba.loc[(ba[col]<1) | (ba[col]>5),col]=np.nan
    
    

In [7]:
ba['recommendation_flag']=(ba['recommended'].str.strip().str.lower().map({'yes':1,'no':0}))

In [8]:
ba['date_flown']=pd.to_datetime(ba['date_flown'],errors='coerce')
ba['year']=ba['date_flown'].dt.year
ba['month']=ba['date_flown'].dt.month

In [9]:
ba['date']=pd.to_datetime(ba['date'],errors='coerce')

In [10]:
text_cols = ['seat_type', 'traveller_type', 'aircraft', 'route', 'place']
for col in text_cols:
    ba[col] = ba[col].astype(str).str.strip().str.title()


In [11]:
ba.isna().sum()

header                   0
author                   0
date                   840
place                    0
content                  0
aircraft                 0
traveller_type           0
seat_type                0
route                    0
date_flown               0
recommended              0
trip_verified            0
rating                 450
seat_comfort            11
cabin_staff_service     11
food_beverages         100
ground_service           4
value_for_money          0
entertainment          457
recommendation_flag      0
year                     0
month                    0
dtype: int64

In [12]:
ba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   header               1324 non-null   object        
 1   author               1324 non-null   object        
 2   date                 484 non-null    datetime64[ns]
 3   place                1324 non-null   object        
 4   content              1324 non-null   object        
 5   aircraft             1324 non-null   object        
 6   traveller_type       1324 non-null   object        
 7   seat_type            1324 non-null   object        
 8   route                1324 non-null   object        
 9   date_flown           1324 non-null   datetime64[ns]
 10  recommended          1324 non-null   object        
 11  trip_verified        1324 non-null   object        
 12  rating               874 non-null    float64       
 13  seat_comfort         1313 non-nul

In [13]:
ba['rating'].describe()

count    874.000000
mean       2.052632
std        1.298329
min        1.000000
25%        1.000000
50%        1.000000
75%        3.000000
max        5.000000
Name: rating, dtype: float64

In [14]:
ba['rating'].value_counts().sort_index()

rating
1.0    439
2.0    160
3.0    130
4.0     80
5.0     65
Name: count, dtype: int64

In [15]:
ba['recommended'].value_counts(normalize=True)*100

recommended
no     56.797583
yes    43.202417
Name: proportion, dtype: float64

In [16]:
ba.groupby('recommended')['rating'].mean()

recommended
no     2.082495
yes    2.013263
Name: rating, dtype: float64

In [17]:
service_cols=['seat_comfort','cabin_staff_service',
       'food_beverages', 'ground_service','value_for_money','entertainment']
ba[service_cols+['rating']].corr()['rating'].sort_values(ascending=False)

rating                 1.000000
entertainment          0.007849
seat_comfort          -0.012244
value_for_money       -0.025074
ground_service        -0.031997
cabin_staff_service   -0.033013
food_beverages        -0.037309
Name: rating, dtype: float64

In [18]:
ba.groupby('seat_type')['rating'].mean().sort_values(ascending=False)

seat_type
First Class        2.344828
Economy Class      2.116279
Business Class     1.967930
Premium Economy    1.906977
Name: rating, dtype: float64

In [19]:
ba.groupby('seat_type')['recommendation_flag'].mean().sort_values(ascending=False)

seat_type
Business Class     0.453094
Premium Economy    0.439394
First Class        0.436782
Economy Class      0.412252
Name: recommendation_flag, dtype: float64

In [20]:
ba.groupby('traveller_type')['rating'].mean().sort_values(ascending=False)

traveller_type
Business          2.164319
Solo Leisure      2.055556
Couple Leisure    2.051546
Family Leisure    1.854701
Nan               1.000000
Name: rating, dtype: float64

In [21]:
ba.groupby('aircraft')['rating'].agg(['count','mean']).sort_values('mean',ascending=False)

,count,mean
aircraft,,
A319 / Boeing 789,1,5.0
A319 / Boeing777,1,5.0
Boeing 787-9 / A380-800,1,5.0
Boeing 777-200 / 787-9 / A319/ 320,1,5.0
Boeing 767-300,1,5.0
...,...,...
Boeing 787-9 / A319,0,NaN
E-190,0,NaN
E170 / A319,0,NaN


In [22]:
ba.groupby('year')['rating'].mean()

year
2015         NaN
2016    2.237410
2017    2.161572
2018    2.072464
2019    1.992857
2020    1.962963
2021    2.250000
2022    1.904762
2023    1.592593
Name: rating, dtype: float64